In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import polygon
import rasterizer

#If testing the libraries this is helpful:
#%load_ext autoreload
#%autoreload 2

In [ ]:
# Generate the n-gon
NGON=7
MAXX=24
MAXY=24
segs=np.array(polygon.gen_poly(NGON,MAXX,MAXY) )

# Rasterize into binary buffer
buf = np.zeros( (30,30))
for nseg in range(len(segs)-1):
    rasterizer.dda1(buf,segs[nseg],segs[nseg+1],do_print=False)

# Visualize the buffer in the notebook
plt.figure(figsize=(5,5))
plt.imshow(buf,cmap='bone_r')

In [ ]:
# Targets or nontargets
ngon=12
maxx=50
maxy=50

buf=np.zeros( (40,40))

range_targ=[24,24]
range_clutter=[20,20]
center_targ=[12,12]
segs=polygon.gen_poly(ngon,range_targ[0],range_targ[1])

segs[:,0] += center_targ[0]

target_present=np.random.randint(2)
if target_present:
    for nseg in range(len(segs)-1):
        rasterizer.dda1(buf,segs[nseg],segs[nseg+1] )
else:
    # For nontarget, largely jitter all endpoints to disconnect ngon
    for nseg in range(len(segs)-1):
        segs[nseg] += np.random.randint(-5,5)
        rasterizer.dda1(buf,segs[nseg],segs[nseg+1] )     
plt.figure(figsize=(2,2))
plt.imshow(buf, cmap='bone_r')
print( target_present )

In [ ]:
# Targets or nontargets (like above), add clutter
ngon=12
maxx=50
maxy=50

buf=np.zeros( (50,50))

range_targ=[24,24]
range_clutter=[20,20]
center_targ=[12,12]
segs=polygon.gen_poly(ngon,range_targ[0],range_targ[1])

segs[:,0] += center_targ[0]

target_present=np.random.randint(2)
if target_present:
    for nseg in range(len(segs)-1):
        rasterizer.dda1(buf,segs[nseg],segs[nseg+1] )
else:
    # For nontarget, largely jitter the endpoints to disconnect ngon
    for nseg in range(len(segs)-1):
        segs[nseg] += np.random.randint(-5,5)
        rasterizer.dda1(buf,segs[nseg],segs[nseg+1] )

# Distribute clutter by choosing centers along x,y space
# walking randomly for each row, then column.
clutter_centers=[ np.random.randint(2,size=2) ]
x=clutter_centers[0][0] 
y=clutter_centers[0][1]
while y<40:
    x=np.random.randint(2)
    while x<24:
        x += np.random.randint(9,12)      # Step along x
        delta_y = np.random.randint(-2,2) # Jitter the y coord a bit
        clutter_centers += [[x,y+delta_y]]
    y += np.random.randint(9,12)          # Go to next row   
    
# For each of these pieces of clutter, generate an ngon
for ndis in range(len(clutter_centers)):
    
    # Generate an ngon
    clutter1=polygon.gen_poly(ngon,range_clutter[0],range_clutter[1])
    seg_start=0
    seg_last=seg_start + np.random.randint(5)
    # If it's a full ngon, randomly choose a subset (partial polygon)
    while (seg_start==0 and seg_last==ngon-1):
        seg_start=np.random.randint(0,ngon-3)
        seg_last=seg_start+4
    
    # Move the clutter to the right position
    clutter1[:,0] += clutter_centers[ndis][0]
    clutter1[:,1] += clutter_centers[ndis][1]
    
    # Render
    for n in range(seg_start,seg_last):
        rasterizer.dda1(buf, clutter1[n], clutter1[n+1],
                       do_print=False,do_flip=True)
        
plt.figure(figsize=(2,2))
plt.imshow(buf, cmap='bone_r')
print( target_present )